In [0]:
import os
import re
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [0]:
from scipy import sparse

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_selection import SelectKBest

In [0]:
with open('Bookstrain.txt','r',encoding='latin1') as f:
  train = f.readlines()

In [0]:
with open('Dvdtest.txt','r',encoding='latin1') as f:
  test = f.readlines()

In [0]:
def clean_text(sentence):
	stopword = stopwords.words('english')
	sent = word_tokenize(sentence)
	sent = [word for word in sent if word.isalpha() and word not in stopword]
	return " ".join(sent)

In [0]:
def preprocess(train_data,test_data):

	label_train = np.zeros(1600)
	label_train[0:800] = 1
	label_test = np.zeros(400)
	label_test[0:200]=1

	train_d,l_train = shuffle(train_data,label_train)
	test_d,l_test = shuffle(test_data,label_test) 

	cor_tr = []

	cor_pos = []
	cor_neg = []

	cor_te = []

	for i in range(1600):
		cor_tr.append(clean_text(train_d[i]))

	for i in range(400):
		cor_te.append(clean_text(test_d[i]))


	return cor_tr,cor_te,l_train,l_test

In [0]:
c_train,c_test,l_train,l_test = preprocess(train,test)

In [0]:
def featureextraction(c_tr,c_te,l_tr,l_te):
	vec = TfidfVectorizer(min_df=3,max_df=0.8,use_idf=True,sublinear_tf=True,stop_words='english')
	tr_tfidf = vec.fit_transform(c_tr)
	te_tfidf = vec.transform(c_te)

	return tr_tfidf,te_tfidf,vec.vocabulary_

In [0]:
tr_tfidf,te_tfidf,vocab = featureextraction(c_train,c_test,l_train,l_test)

In [0]:
def get_freq(tr_d,tr_l,te_d,te_l,word):

	cp=0.0
	cn=0.0

	for i in range(0,1600):
		words = tr_d[i].split()
		if(word in words):
			if(tr_l[i]==1):
				cp+=1
			else:
				cn+=1

	for i in range(0,400):
		words = te_d[i].split()
		if(word in words):
			if(te_l[i]==1):
				cp+=1
			else:
				cn+=1

	return (cp+cn)/2

In [0]:
def cal_word_freq(tr_d,te_d,tr_l,te_l,vocab):

	freq_c = []
	
	for word in vocab:
		cou = get_freq(tr_d,tr_l,te_d,te_l,word)
		freq_c.append([word,cou])

	org_freq = sorted(freq_c,key=lambda x:x[1],reverse=True)

	return org_freq


In [0]:
freq = cal_word_freq(c_train,c_test,l_train,l_test,vocab)

In [0]:
k = 5000

In [0]:
kfea = []

In [0]:
for i in range(k):
  kfea.append(freq[i][0])

In [0]:
tr_vecs = np.zeros((1600,k))

In [0]:
for i in range(0,1600):
  sents = c_train[i].split()
  for j in range(len(sents)):
    if(sents[j] in kfea):
      tr_vecs[i][j]=1

In [0]:
te_vecs = np.zeros((400,k))

In [0]:
for i in range(0,400):
  sents = c_test[i].split()
  for j in range(len(sents)):
    if(sents[j] in kfea):
      te_vecs[i][j]=1

In [0]:
tra = sparse.csr_matrix(tr_vecs)

In [0]:
tea = sparse.csr_matrix(te_vecs)

In [75]:
lr = LogisticRegression()
lr.fit(tra,l_train)
lr_p = lr.predict(tea)
lr_a = accuracy_score(lr_p,l_test)
lr_f1 = f1_score(lr_p,l_test)
lr_cm = confusion_matrix(lr_p,l_test)
print("Accuracy :",lr_a)
print("F1 score :",lr_f1)
print(lr_cm)

Accuracy : 0.4725
F1 score : 0.4575835475578406
[[100 111]
 [100  89]]


In [86]:
lrb  = BaggingClassifier(base_estimator=lr,random_state=0,max_samples=0.8,n_estimators=10)
lrb.fit(tra,l_train)
lrbp = lrb.predict(tea)
lrba = lrb.score(tea,l_test)
lrbcm = confusion_matrix(lrbp,l_test)
lrbf1 = f1_score(lrbp,l_test)
print("Accuracy :",lrba)
print("F1 score :",lrbf1)
print(lrbcm)

Accuracy : 0.4875
F1 score : 0.48621553884711777
[[ 98 103]
 [102  97]]


In [77]:
rb = svm.SVC(gamma=0.9)
rb.fit(tra,l_train)
rbp = rb.predict(tea)
rba = accuracy_score(rbp,l_test)
rbcm = confusion_matrix(rbp,l_test)
rbf1 = f1_score(rbp,l_test)
print("The SVM (rbf) Accuracy is: ",rba)
print("The F1 score is: ",rbf1)
print("The confusion matrix is:")
print(rbcm)


The SVM (rbf) Accuracy is:  0.49
The F1 score is:  0.6494845360824743
The confusion matrix is:
[[  7  11]
 [193 189]]


In [81]:
rbb  = BaggingClassifier(base_estimator=rb,random_state=0,max_samples=0.8,n_estimators=13)
rbb.fit(tra,l_train)
rbbp = rbb.predict(tea)
rbba = accuracy_score(rbbp,l_test)
rbbcm = confusion_matrix(rbbp,l_test)
rbbf1 = f1_score(rbbp,l_test)
print("Accuracy(SVM) Bagging:",rbba)
print("F1 score :",rbbf1)
print(rbbcm)

Accuracy(SVM) Bagging: 0.495
F1 score : 0.6505190311418685
[[ 10  12]
 [190 188]]


In [87]:
ls = svm.SVC(gamma=0.9,kernel='linear')
ls.fit(tra,l_train)
lsp = ls.predict(tea)
lsa = accuracy_score(lsp,l_test)
lscm = confusion_matrix(lsp,l_test)
lsf1 = f1_score(lsp,l_test)
print("The SVM (rbf) Accuracy is: ",lsa)
print("The F1 score is: ",lsf1)
print("The confusion matrix is:")
print(lscm)


The SVM (rbf) Accuracy is:  0.4975
The F1 score is:  0.513317191283293
The confusion matrix is:
[[ 93  94]
 [107 106]]


In [96]:
lsb  = BaggingClassifier(base_estimator=ls,random_state=0,max_samples=0.6,n_estimators=5)
lsb.fit(tra,l_train)
lsbp = lsb.predict(tea)
lsba = accuracy_score(lsbp,l_test)
lsbcm = confusion_matrix(lsbp,l_test)
lsbf1 = f1_score(lsbp,l_test)
print("Accuracy(Linear SVM) Bagging:",lsba)
print("F1 score :",lsbf1)
print(lsbcm)

Accuracy(Linear SVM) Bagging: 0.53
F1 score : 0.5566037735849056
[[ 94  82]
 [106 118]]


In [97]:
dt = DecisionTreeClassifier()
dt.fit(tra,l_train)
dtp = dt.predict(tea)
dta = accuracy_score(dtp,l_test)
dtcm = confusion_matrix(dtp,l_test)
dtf1 = f1_score(dtp,l_test)
print("The DT Accuracy is: ",dta)
print("The F1 score is: ",dtf1)
print("The confusion matrix is:")
print(dtcm)

The DT Accuracy is:  0.4975
The F1 score is:  0.4962406015037594
The confusion matrix is:
[[100 101]
 [100  99]]


In [98]:
dtb  = BaggingClassifier(base_estimator=dt,random_state=0,max_samples=0.6,n_estimators=5)
dtb.fit(tra,l_train)
dtbp = dtb.predict(tea)
dtba = accuracy_score(dtbp,l_test)
dtbcm = confusion_matrix(dtbp,l_test)
dtbf1 = f1_score(dtbp,l_test)
print("Accuracy(DT) Bagging:",dtba)
print("F1 score :",dtbf1)
print(dtbcm)

Accuracy(DT) Bagging: 0.535
F1 score : 0.535
[[107  93]
 [ 93 107]]


In [99]:
mb = MultinomialNB()
mb.fit(tra,l_train)
mbp = mb.predict(tea)
mba = accuracy_score(mbp,l_test)
mbcm = confusion_matrix(mbp,l_test)
mbf1 = f1_score(mbp,l_test)
print("The MNB Accuracy is: ",mba)
print("The F1 score is: ",mbf1)
print("The confusion matrix is:")
print(mbcm)

The MNB Accuracy is:  0.5175
The F1 score is:  0.6584070796460177
The confusion matrix is:
[[ 21  14]
 [179 186]]


In [124]:
mbb  = BaggingClassifier(base_estimator=mb,random_state=0,max_samples=0.6,n_estimators=13)
mbb.fit(tra,l_train)
mbbp = mbb.predict(tea)
mbba = accuracy_score(mbbp,l_test)
mbbcm = confusion_matrix(mbbp,l_test)
mbbf1 = f1_score(mbbp,l_test)
print("Accuracy(MNB) Bagging:",mbba)
print("F1 score :",mbbf1)
print(mbbcm)

Accuracy(DT) Bagging: 0.515
F1 score : 0.6608391608391608
[[ 17  11]
 [183 189]]


In [125]:
be = BernoulliNB()
be.fit(tra,l_train)
bep = be.predict(tea)
bea = accuracy_score(bep,l_test)
becm = confusion_matrix(bep,l_test)
bef1 = f1_score(bep,l_test)
print("The Ber Accuracy is: ",bea)
print("The F1 score is: ",bef1)
print("The confusion matrix is:")
print(becm)

The Ber Accuracy is:  0.4925
The F1 score is:  0.6588235294117647
The confusion matrix is:
[[  1   4]
 [199 196]]


In [159]:
beb  = BaggingClassifier(base_estimator=be,random_state=0,max_samples=0.8,n_estimators=9,max_features=0.8)
beb.fit(tra,l_train)
bebp = beb.predict(tea)
beba = accuracy_score(bebp,l_test)
bebcm = confusion_matrix(bebp,l_test)
bebf1 = f1_score(bebp,l_test)
print("Accuracy(BER) Bagging:",beba)
print("F1 score :",bebf1)
print(bebcm)

Accuracy(BER) Bagging: 0.525
F1 score : 0.6631205673758865
[[ 23  13]
 [177 187]]


In [164]:
lrrs  = BaggingClassifier(base_estimator=lr,random_state=0,max_features=0.8,n_estimators=10)
lrrs.fit(tra,l_train)
lrrsp = lrrs.predict(tea)
lrrsa = lrrs.score(tea,l_test)
lrrscm = confusion_matrix(lrrsp,l_test)
lrrsf1 = f1_score(lrrsp,l_test)
print("Accuracy (RS):",lrrsa)
print("F1 score :",lrrsf1)
print(lrrscm)

Accuracy (RS): 0.4925
F1 score : 0.46997389033942555
[[107 110]
 [ 93  90]]


In [172]:
rbrs  = BaggingClassifier(base_estimator=rb,random_state=0,max_features=0.8,n_estimators=7)
rbrs.fit(tra,l_train)
rbrsp = rbrs.predict(tea)
rbrsa = accuracy_score(rbrsp,l_test)
rbrscm = confusion_matrix(rbrsp,l_test)
rbrsf1 = f1_score(rbrsp,l_test)
print("Accuracy(SVM) RS:",rbrsa)
print("F1 score :",rbrsf1)
print(rbrscm)

Accuracy(SVM) RS: 0.5025
F1 score : 0.6514886164623468
[[ 15  14]
 [185 186]]


In [194]:
lrs  = BaggingClassifier(base_estimator=ls,random_state=0,max_features=0.99,n_estimators=7)
lrs.fit(tra,l_train)
lrsp = lrs.predict(tea)
lrsa = accuracy_score(lrsp,l_test)
lrscm = confusion_matrix(lrsp,l_test)
lrsf1 = f1_score(lrsp,l_test)
print("Accuracy(Linear SVM) RS:",lrsa)
print("F1 score :",lrsf1)
print(lrscm)

Accuracy(Linear SVM) RS: 0.5
F1 score : 0.5238095238095238
[[ 90  90]
 [110 110]]


In [195]:
dtrs  = BaggingClassifier(base_estimator=dt,random_state=0,max_features=0.6,n_estimators=5)
dtrs.fit(tra,l_train)
dtrsp = dtrs.predict(tea)
dtrsa = accuracy_score(dtrsp,l_test)
dtrscm = confusion_matrix(dtrsp,l_test)
dtrsf1 = f1_score(dtrsp,l_test)
print("Accuracy(DT) RS:",dtrsa)
print("F1 score :",dtrsf1)
print(dtrscm)

Accuracy(DT) RS: 0.5325
F1 score : 0.5336658354114714
[[106  93]
 [ 94 107]]


In [228]:
mbrs  = BaggingClassifier(base_estimator=mb,random_state=0,max_features=0.7,n_estimators=14)
mbrs.fit(tra,l_train)
mbrsp = mbrs.predict(tea)
mbrsa = accuracy_score(mbrsp,l_test)
mbrscm = confusion_matrix(mbrsp,l_test)
mbrsf1 = f1_score(mbrsp,l_test)
print("Accuracy(MNB) RS:",mbrsa)
print("F1 score :",mbrsf1)
print(mbrscm)

Accuracy(MNB) RS: 0.52
F1 score : 0.6619718309859155
[[ 20  12]
 [180 188]]


In [229]:
bers  = BaggingClassifier(base_estimator=be,random_state=0,n_estimators=9,max_features=0.8)
bers.fit(tra,l_train)
bersp = bers.predict(tea)
bersa = accuracy_score(bersp,l_test)
berscm = confusion_matrix(bersp,l_test)
bersf1 = f1_score(bersp,l_test)
print("Accuracy(BER) RS :",bersa)
print("F1 score :",bersf1)
print(berscm)

Accuracy(BER) RS : 0.505
F1 score : 0.6621160409556314
[[  8   6]
 [192 194]]


In [232]:
lrbo  = AdaBoostClassifier(base_estimator=lr,random_state=0,n_estimators=10)
lrbo.fit(tra,l_train)
lrbop = lrbo.predict(tea)
lrboa = lrbo.score(tea,l_test)
lrbocm = confusion_matrix(lrbop,l_test)
lrbof1 = f1_score(lrbop,l_test)
print("Accuracy (Boosting):",lrboa)
print("F1 score :",lrbof1)
print(lrbocm)

Accuracy (Boosting): 0.5175
F1 score : 0.4769647696476965
[[119 112]
 [ 81  88]]


In [240]:
rbbo  = AdaBoostClassifier(base_estimator=rb,random_state=0,n_estimators=10,algorithm='SAMME',learning_rate=0.1)
rbbo.fit(tra,l_train)
rbbop = rbbo.predict(tea)
rbboa = accuracy_score(rbbop,l_test)
rbbocm = confusion_matrix(rbbop,l_test)
rbbof1 = f1_score(rbbop,l_test)
print("Accuracy(SVM) Boosting:",rbboa)
print("F1 score :",rbbof1)
print(rbbocm)

Accuracy(SVM) Boosting: 0.4975
F1 score : 0.01951219512195122
[[197 198]
 [  3   2]]


In [236]:
lbo  = AdaBoostClassifier(base_estimator=ls,random_state=0,n_estimators=7,algorithm='SAMME')
lbo.fit(tra,l_train)
lbop = lbo.predict(tea)
lboa = accuracy_score(lbop,l_test)
lbocm = confusion_matrix(lbop,l_test)
lbof1 = f1_score(lbop,l_test)
print("Accuracy(Linear SVM) Boosting:",lboa)
print("F1 score :",lbof1)
print(lbocm)

Accuracy(Linear SVM) Boosting: 0.5025
F1 score : 0.5273159144893111
[[ 90  89]
 [110 111]]


In [0]:
mbbo  = BaggingClassifier(base_estimator=mb,random_state=0,n_estimators=14)
mbbo.fit(tra,l_train)
mbbop = mbbo.predict(tea)
mbboa = accuracy_score(mbbop,l_test)
mbbocm = confusion_matrix(mbbop,l_test)
mbbof1 = f1_score(mbbop,l_test)
print("Accuracy(MNB) Boosting:",mbboa)
print("F1 score :",mbbof1)
print(mbbocm)